In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, utils, models
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
from PIL import Image
import random

from model import initialize_model, set_parameter_requires_grad

In [2]:
model_ft = initialize_model("delf", 2000, 2, use_pretrained=True)

In [5]:
list(model_ft.children())[2]

AttentionPool(
  (conv1): Conv2d(1664, 512, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))
  (activ): Softplus(beta=1, threshold=20)
)

In [76]:
from initialize_delf import load_densenet
model_path = '/home/kylecshan/saved/06_01_2019/classification/densenet169/model1_epoch9.pth'
output_dim = 1664

model = load_densenet(model_path, 2000)

In [75]:
list(model.module.children())[:-2]

[Sequential(
   (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu0): ReLU(inplace)
   (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (denseblock1): _DenseBlock(
     (denselayer1): _DenseLayer(
       (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu1): ReLU(inplace)
       (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu2): ReLU(inplace)
       (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     )
     (denselayer2): _DenseLayer(
       (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu1): ReLU(inplace)
       (conv1): Conv2d(96, 128, kern

In [3]:
def load_model(model_name, model_path):
         
    # Initialize the model for this run
    model_ft = initialize_model(model_name, 2000, 0, use_pretrained=True)
        
    features = list(model_ft.children())[:-1]
    features.append(nn.ReLU(inplace=True))
    features.append(nn.AdaptiveAvgPool2d((1, 1)))
    
    model = nn.Sequential(*features)
    model_dict = model.state_dict()
    load_dict = torch.load(model_path)

    temp_dict = dict(load_dict)
    for k, v in temp_dict.items():
        s = list(k)
        del s[:15]
        s.insert(0, "0") 
        newk = "".join(s)
        load_dict[newk] = load_dict.pop(k)
        
    model_dict = {k: v for k, v in load_dict.items() if k in model_dict}
    model.load_state_dict(model_dict)

    return model

In [4]:

model_path = '/home/kylecshan/saved/05_16_2019/model815.pth'
model_name = "densenet169"
output_dim = 1664
batch_size = 256
num_classes = 2000

#model = load_model(model_name, model_path)

In [8]:
model = load_model(model_name, model_path)

In [9]:
model

Sequential(
  (0): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
  

In [13]:
class AttentionPool(torch.nn.Module):
    def __init__(self, d_in):
        super(AttentionPool, self).__init__()
        self.conv1 = torch.nn.Conv2d(d_in, 512, 1)
        self.conv2 = torch.nn.Conv2d(512, 1, 1)
        self.activ2 = torch.nn.Softplus()
        
    def forward(self, x):
        scores = self.conv2(self.conv1(x))
        weights = self.activ2(scores)
        wtd_feat = x * weights
        out_feat = torch.mean(wtd_feat, dim=(2,3))

In [14]:
delf_model = nn.Sequential(*list(model.children())[:-1],
                           AttentionPool(output_dim))

In [23]:
set_parameter_requires_grad(delf_model, 2)

In [4]:
asdf = nn.Conv2d(1664, 512, 1)
asdf.weight.size()

torch.Size([512, 1664, 1, 1])

In [8]:
model_ft = models.densenet169(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(1664, 2000)

In [11]:
torch.load(model_path).keys()

odict_keys(['module.features.conv0.weight', 'module.features.norm0.weight', 'module.features.norm0.bias', 'module.features.norm0.running_mean', 'module.features.norm0.running_var', 'module.features.norm0.num_batches_tracked', 'module.features.denseblock1.denselayer1.norm1.weight', 'module.features.denseblock1.denselayer1.norm1.bias', 'module.features.denseblock1.denselayer1.norm1.running_mean', 'module.features.denseblock1.denselayer1.norm1.running_var', 'module.features.denseblock1.denselayer1.norm1.num_batches_tracked', 'module.features.denseblock1.denselayer1.conv1.weight', 'module.features.denseblock1.denselayer1.norm2.weight', 'module.features.denseblock1.denselayer1.norm2.bias', 'module.features.denseblock1.denselayer1.norm2.running_mean', 'module.features.denseblock1.denselayer1.norm2.running_var', 'module.features.denseblock1.denselayer1.norm2.num_batches_tracked', 'module.features.denseblock1.denselayer1.conv2.weight', 'module.features.denseblock1.denselayer2.norm1.weight', 'm

In [16]:
model_ft = models.densenet169(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, num_classes)

features = list(model_ft.children())[:-1]
features.append(nn.ReLU(inplace=True))
features.append(nn.AdaptiveAvgPool2d((1, 1)))
features.append(list(model_ft.children())[-1])

model = nn.Sequential(*features)
model_dict = model.state_dict()
load_dict = torch.load(model_path)

temp_dict = dict(load_dict)
for k, v in temp_dict.items():
    s = list(k)
    if s[:17] == list("module.classifier"):
        del s[:17]
        s.insert(0, "3")
        newk = "".join(s)
        load_dict[newk] = load_dict.pop(k)
    else:
        del s[:15]
        s.insert(0, "0") 
        newk = "".join(s)
        load_dict[newk] = load_dict.pop(k)

print(load_dict.keys())
model_dict = {k: v for k, v in load_dict.items() if k in model_dict}
model.load_state_dict(model_dict)

odict_keys(['0.conv0.weight', '0.norm0.weight', '0.norm0.bias', '0.norm0.running_mean', '0.norm0.running_var', '0.norm0.num_batches_tracked', '0.denseblock1.denselayer1.norm1.weight', '0.denseblock1.denselayer1.norm1.bias', '0.denseblock1.denselayer1.norm1.running_mean', '0.denseblock1.denselayer1.norm1.running_var', '0.denseblock1.denselayer1.norm1.num_batches_tracked', '0.denseblock1.denselayer1.conv1.weight', '0.denseblock1.denselayer1.norm2.weight', '0.denseblock1.denselayer1.norm2.bias', '0.denseblock1.denselayer1.norm2.running_mean', '0.denseblock1.denselayer1.norm2.running_var', '0.denseblock1.denselayer1.norm2.num_batches_tracked', '0.denseblock1.denselayer1.conv2.weight', '0.denseblock1.denselayer2.norm1.weight', '0.denseblock1.denselayer2.norm1.bias', '0.denseblock1.denselayer2.norm1.running_mean', '0.denseblock1.denselayer2.norm1.running_var', '0.denseblock1.denselayer2.norm1.num_batches_tracked', '0.denseblock1.denselayer2.conv1.weight', '0.denseblock1.denselayer2.norm2.wei

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [17]:
model.state_dict().keys()

odict_keys(['0.conv0.weight', '0.norm0.weight', '0.norm0.bias', '0.norm0.running_mean', '0.norm0.running_var', '0.norm0.num_batches_tracked', '0.denseblock1.denselayer1.norm1.weight', '0.denseblock1.denselayer1.norm1.bias', '0.denseblock1.denselayer1.norm1.running_mean', '0.denseblock1.denselayer1.norm1.running_var', '0.denseblock1.denselayer1.norm1.num_batches_tracked', '0.denseblock1.denselayer1.conv1.weight', '0.denseblock1.denselayer1.norm2.weight', '0.denseblock1.denselayer1.norm2.bias', '0.denseblock1.denselayer1.norm2.running_mean', '0.denseblock1.denselayer1.norm2.running_var', '0.denseblock1.denselayer1.norm2.num_batches_tracked', '0.denseblock1.denselayer1.conv2.weight', '0.denseblock1.denselayer2.norm1.weight', '0.denseblock1.denselayer2.norm1.bias', '0.denseblock1.denselayer2.norm1.running_mean', '0.denseblock1.denselayer2.norm1.running_var', '0.denseblock1.denselayer2.norm1.num_batches_tracked', '0.denseblock1.denselayer2.conv1.weight', '0.denseblock1.denselayer2.norm2.wei

In [ ]:
model_ft.state_dict().keys